# Setup

In [14]:
# %pip install -r requirements.txt

# Environment Variables

In [15]:
# Get environment variables

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Google Colab Auth

In [16]:
# Check if running in Google Colab

import sys

# If running in Colab, use the permissions of the currently authenticated user
if "google.colab" in sys.modules:
	print('Running in Google Colab')

	from google.colab import auth

	auth.authenticate_user()

# If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file
else:
	print("Running locally")

Running locally


# Utilities


In [17]:
# Import the utils module

from utils import *

# RAG Engine

In [18]:
import os

import vertexai
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool

PROJECT_ID = os.environ["PROJECT_ID"]
LOCATION = os.environ["REGION"]
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Currently supports Google first-party embedding models
EMBEDDING_MODEL = "publishers/google/models/text-embedding-004"
embedding_model_config = rag.EmbeddingModelConfig(publisher_model=EMBEDDING_MODEL)

rag_corpus = rag.create_corpus(display_name="10k-docs", embedding_model_config=embedding_model_config)
rag.list_corpora()



ListRagCorporaPager<rag_corpora {
  name: "projects/era4-447717/locations/us-central1/ragCorpora/4611686018427387904"
  display_name: "10k-docs"
  create_time {
    seconds: 1739305244
    nanos: 613044000
  }
  update_time {
    seconds: 1739305244
    nanos: 613044000
  }
  rag_embedding_model_config {
    vertex_prediction_endpoint {
      endpoint: "projects/era4-447717/locations/us-central1/publishers/google/models/text-embedding-004"
    }
  }
  rag_vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/era4-447717/locations/us-central1/publishers/google/models/text-embedding-004"
      }
    }
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/era4-447717/locations/us-central1/publishers/google/models/text-embedding-004"
      }
    }
  }
}
rag_corpora {
  na

In [19]:
rag_file_lyft = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="docs/10k/pdf/lyft.pdf",
    display_name="lyft.pdf",
    description="Lyft 10K for 2023",
)

rag_file_uber = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="docs/10k/pdf/uber.pdf",
    display_name="uber.pdf",
    description="Uber 10K for 2023",
)


In [22]:
# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    text="What was Uber's revenue for 2023?",
    similarity_top_k=10,  # Optional
    vector_distance_threshold=0.5,  # Optional
)
# print(response)

# Optional: The retrieved context can be passed to any SDK or model generation API to generate final results.
context = " ".join([context.text for context in response.contexts.contexts]).replace("\n", "")
print(context)

41254


In [23]:
# Create a tool for the RAG Corpus
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(
        source=rag.VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        ),
    )
)

In [30]:
# Load tool into Gemini model
rag_gemini_model = GenerativeModel(os.environ["GOOGLE_LLM_MODEL_NAME"], tools=[rag_retrieval_tool],)

In [35]:
from IPython.display import Markdown

questions = [
	"What is the annual revenue of Uber?",
	"What is the annual revenue of Lyft?",
	"How does Uber's revenue compare to Lyft's revenue?",
	"Summarize Lyft's risk factors",
	]

for question in questions:
    print(question)
    response = rag_gemini_model.generate_content(question)
	#display(Markdown(response.text))
    print(response.text)


What is the annual revenue of Uber?
Uber's revenue for 2023 was $37.3 billion, a 17% increase year-over-year.
What is the annual revenue of Lyft?
Lyft's revenue for the year ended December 31, 2023, was $4,403,589,000. In 2022, the revenue was $4,095,135,000, and in 2021, it was $3,208,323,000.
How does Uber's revenue compare to Lyft's revenue?
Uber's revenue for 2023 was $37.3 billion, a 17% increase year-over-year. Lyft's revenue for 2023 was $4.4 billion, an 8% increase compared to 2022.

Summarize Lyft's risk factors
Lyft's business operations are subject to numerous risks, factors, and uncertainties, many of which are outside of their control, that could harm actual results. These risks include:

**General Economic Factors:**
General macroeconomic conditions, the impact of the COVID-19 pandemic, natural disasters, economic downturns, public health crises, or political crises.

**Operational Factors:**
Limited operating history, inability to achieve or maintain profitability, compe